In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import textwrap
import json
import os
import plotly.express as px
import plotly.graph_objects as go
import folium as fl

# Exercise 1

In [ ]:
data_weather = pd.read_csv('../data/raw/weather/weather.csv', sep = "\t")
data_corona_de = pd.read_csv("../data/raw/corona/de_corona.csv", sep = "\t")
with open("../data/raw/metadata/de_metadata.json") as f:
    data_meta_de = json.load(f)
with open("../data/raw/shapefiles/de.geojson") as f:
    data_geo = json.load(f)

In [ ]:
print(data_weather.shape)
data_weather.head()

In [ ]:
print(list(data_weather.columns))
np.unique(data_weather["iso3166-2"])

Sanity check

In [ ]:
count = 0
for i in data_weather:
    #print(sum(pd.isnull(data_weather[i])))
    if sum(pd.isnull(data_weather[i])) != 0:
        count += 1
print(count)

Filtering for germany

In [ ]:
de_codes = ['DE-BB', 'DE-BE', 'DE-BW', 'DE-BY', 'DE-HB', 'DE-HE', 'DE-HH', 'DE-MV', 'DE-NI', 'DE-NW', 
            'DE-RP', 'DE-SH', 'DE-SL', 'DE-SN', 'DE-ST', 'DE-TH']
mask = np.isin(data_weather, de_codes) 
data_weather_de = data_weather[mask]

print(data_weather_de.shape)
np.unique(data_weather_de["iso3166-2"])

In [ ]:
# saving the filtered dataset for germany
def saving_csv(file, name):
    file.to_csv(f'../data/interim/weather_{name}.csv')

In [ ]:
saving_csv(data_weather_de, "germany")

In [ ]:
print(data_corona_de.shape)
data_corona_de.head()

# min, mean, median, and max of regions

In [ ]:
# masking for different regions
region_names = ['DE_BB', 'DE_BE', 'DE_BW', 'DE_BY', 'DE_HB', 'DE_HE', 'DE_HH', 'DE_MV', 'DE_NI', 'DE_NW', 
            'DE_RP', 'DE_SH', 'DE_SL', 'DE_SN', 'DE_ST', 'DE_TH']
for codes, names in zip(de_codes, de_codes):
    mask = np.isin(data_weather_de, codes)
    region_data = data_weather_de[mask]
    saving_csv(region_data, names)

In [ ]:
# filter for numerical columns, they are from index 2 and onwards
weather_columns = list(data_weather.columns)
print(weather_columns)
weather_numerical_columns = weather_columns[2:9]
weather_numerical_columns

In [ ]:
dict_sum_numeric = {}
for region in de_codes:
    data = pd.read_csv(f'../data/interim/weather_{region}.csv')
    dict_sum_numeric.update({region:{}})
    for col in range(2,len(list(data_weather.columns))):
        dict_sum_numeric[region].update({list(data_weather.columns)[col]:[min(data[list(data_weather.columns)[col]]), 
                                                             data[list(data_weather.columns)[col]].mean(), 
                                                             data[list(data_weather.columns)[col]].median(), 
                                                            max(data[list(data_weather.columns)[col]])]})
print(dict_sum_numeric["DE-BB"])

In [ ]:
fig = go.FigureWidget(data=go.Bar(y=dict_sum_numeric["DE-BB"]["WindSpeed"], x = ["min", "mean", "median", "max"]))
fig

# Exercise 2

In [ ]:
#create dictionary to convert the region code into the region name
dic_convert = {data_meta_de["country_metadata"][i]["iso3166-2_name_en"].replace("Ã¼", "ü"):
               data_meta_de["country_metadata"][i]["iso3166-2_code"]
              for i in range(len(data_meta_de["country_metadata"]))
              }
#insert a column into corona dataframe with the region name as well
data_corona_de["region"] = data_corona_de["region_code"].map(dic_convert)

#create a new dataframe having region code and the confirmed cases
data_corona_region = data_corona_de.groupby(by = "region")["confirmed_addition"].sum().reset_index()

#create a dictionary having the total population for each region
population_map = {data_meta_de["country_metadata"][i]["iso3166-2_code"]:
                  data_meta_de["country_metadata"][i]["population"] 
                  for i in range(len(data_meta_de["country_metadata"]))
                 }
#insert a column population into data coron by region
data_corona_region["population"] = data_corona_region["region"].map(population_map)

#insert a coulm that cases/population
data_corona_region["relation"] = data_corona_region["confirmed_addition"]/data_corona_region["population"]*100

data_corona_region

In [ ]:
#create the folium.Choropleth
m = fl.Map(
location = [51.3, 10.3],
    zoom_start = 6,)

fl.Choropleth(
    geo_data = data_geo,
    name = "cases",
    data = data_corona_region,
    columns = ["region", "relation"],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd",
    fill_opacity = .6,
    line_opacity = .5,
    legend_name = "Number of Cases",
).add_to(m)

m